<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8P%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본			   G_inc_so_model_노이2.pt
 산업분류자동화		   G_inc_so_model_노이.pt
'Colab Notebooks'	  'My Drive'
 G_inc_again_so_model.pt   Screenshot_20220401-001411_Chrome.jpg


In [3]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/P_spell_check.csv', sep = ",", encoding = "UTF-8")

## 데이터 전처리

In [4]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000241,P,85,851,초등학교 관할 교육청 인가를 받아 중학교 교육을 위한 서비스
1,id_0000323,P,85,851,초등학교 관할 교육청인가를 받아 초등교육서비스
2,id_0000905,P,85,851,초등학교 관할지 녘 교육청의 인가를 받아 초등 전 학년을 교육
3,id_0001730,P,85,851,유치원 관할 지역교육청의 인가를 받아 취학 전 아동교육
4,id_0002025,P,85,851,초등학교 관할 지역교육청의 인가를 받아 초등교육
...,...,...,...,...,...
46605,id_0995113,P,85,857,사업장 교육 관련 평가
46606,id_0998362,P,85,857,영업장에서 고객 대상으로 입시상담
46607,id_0998364,P,85,857,사업장 교육자문 서비스
46608,id_0999233,P,85,857,사무실에서 일반인 대상 교육지원


In [5]:
df['digit_3'].value_counts().sort_index()

851     3544
852     1333
853      366
854       67
855    16862
856    23521
857      917
Name: digit_3, dtype: int64

## **# 데이터 증분**

In [6]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [7]:
df0=df.loc[(df['digit_3'] == 851)]  
df1=df.loc[(df['digit_3'] == 852)] #*3
df2=df.loc[(df['digit_3'] == 853)]  #*10
df3=df.loc[(df['digit_3'] == 854)]  #*50
df4=df.loc[(df['digit_3'] == 855)]   
df5=df.loc[(df['digit_3'] == 856)]   
df6=df.loc[(df['digit_3'] == 857)]    #*3

In [8]:
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)
df4=df4.reset_index(drop=True)
df5=df5.reset_index(drop=True)
df6=df6.reset_index(drop=True)

In [9]:
df_1=pd.concat([df0, df4, df5]) # *1
df_3=pd.concat([df1, df6]) # *3
df_10=pd.concat([df2]) # *10 묶음
df_50=pd.concat([df3]) # *50

In [10]:
df_1=df_1.reset_index(drop=True)
df_3=df_3.reset_index(drop=True)
df_10=df_10.reset_index(drop=True)
df_50=df_50.reset_index(drop=True)

In [11]:
df1.head()

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000406,P,85,852,중학교에서 중학생을 대상으로 고등 진학을 위한 교육서비스
1,id_0000550,P,85,852,인문계 고등학교 인문계 교육 대학 진학을 위한 교육서비스
2,id_0000599,P,85,852,인문계 고등학교 인문계 교육 대학 진학을 위한 교육서비스
3,id_0000711,P,85,852,학교에서 중학생 대상으로 중등교육
4,id_0000816,P,85,852,중학교에서 중학생을 대상으로 고등학교 진학을 위한 교육서비스


In [12]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df_3['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=['p'+str(df_3['AI_id'][n]), 'p'+str(df_3['AI_id'][n]), 'p'+str(df_3['AI_id'][n])]
  digit_1_list=[df_3['digit_1'][n], df_3['digit_1'][n], df_3['digit_1'][n]]
  digit_2_list=[df_3['digit_2'][n], df_3['digit_2'][n], df_3['digit_2'][n]]
  digit_3_list=[df_3['digit_3'][n], df_3['digit_3'][n], df_3['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_3_to_insert = pd.DataFrame(data_to_insert)
  df_3= df_3.append(df_3_to_insert, ignore_index=True)
  n=n+1

In [13]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_10['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=9)
  id_list=[str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n])]
  digit_1_list=[df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n]]
  digit_2_list=[df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n]]
  digit_3_list=[df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_10_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_10_to_insert, ignore_index=True)
  n=n+1

df_10=temp

In [14]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_50['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=49)
  id_list=[str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n])]
  digit_1_list=[df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n]]
  digit_2_list=[df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n]]
  digit_3_list=[df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_50_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_50_to_insert, ignore_index=True)
  n=n+1

df_50=temp

기존 데이터 합치기

In [15]:
df=pd.concat([df_1, df_3, df_10, df_50])

In [16]:
df['digit_3'].value_counts().sort_index()

851.0     3544
852.0     5332
853.0     3660
854.0     3350
855.0    16862
856.0    23521
857.0     3668
Name: digit_3, dtype: int64

In [17]:
df.head()

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000241,P,85.0,851.0,초등학교 관할 교육청 인가를 받아 중학교 교육을 위한 서비스
1,id_0000323,P,85.0,851.0,초등학교 관할 교육청인가를 받아 초등교육서비스
2,id_0000905,P,85.0,851.0,초등학교 관할지 녘 교육청의 인가를 받아 초등 전 학년을 교육
3,id_0001730,P,85.0,851.0,유치원 관할 지역교육청의 인가를 받아 취학 전 아동교육
4,id_0002025,P,85.0,851.0,초등학교 관할 지역교육청의 인가를 받아 초등교육


In [18]:
df.to_csv('df_P_inc.csv', index=False, encoding='utf-8-sig')